In [1]:
!pip install transformers
!pip install datasets transformers[sentencepiece] sacrebleu
import pandas as pd
import numpy as np
from datasets import load_dataset
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


In [2]:
dataset_path = "cfilt/iitb-english-hindi"
train_dataset = load_dataset(dataset_path, split="train[:1000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [3]:
english_sentences = [item['en'] for item in train_dataset['translation']]
hindi_sentences = [item['hi'] for item in train_dataset['translation']]

In [4]:
from transformers import pipeline

pipe = pipeline("translation", model="facebook/mbart-large-50-many-to-many-mmt")

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [6]:
model= MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to('cuda')

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

In [7]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="en_XX", tgt_lang='hi_IN',max_length=400)

In [8]:
translatedEngHin = []
for (index, (eng, hin)) in enumerate(zip(english_sentences, hindi_sentences)):
    print(index+1)
    print("English:", eng)
    prediction = translator(eng)
    translation = prediction[0]['translation_text']
    print("Translated: ", translation)
    print("Original: ", hin)
    print("------------------------------")
    translatedEngHin.append(translation)

1
English: Give your application an accessibility workout
Translated:  अपने अनुप्रयोग को एक अभिगमता प्रशिक्षण दें
Original:  अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
------------------------------
2
English: Accerciser Accessibility Explorer
Translated:  एक्सरसीसर एक्सप्लोरर
Original:  एक्सेर्साइसर पहुंचनीयता अन्वेषक
------------------------------
3
English: The default plugin layout for the bottom panel
Translated:  निचले पैनल के लिए डिफ़ॉल्ट प्लगइन सजावट
Original:  निचले पटल के लिए डिफोल्ट प्लग-इन खाका
------------------------------
4
English: The default plugin layout for the top panel
Translated:  शीर्ष पैनल के लिए डिफ़ॉल्ट प्लगइन सजावट
Original:  ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
------------------------------
5
English: A list of plugins that are disabled by default
Translated:  प्लगइन की सूची जिसे डिफ़ॉल्ट द्वारा अक्षम किया गया है
Original:  उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
------------------------------
6
English: Highlight duration
Tr

In [9]:
comaparision = pd.DataFrame({'Original': english_sentences, 'Translated': translatedEngHin})
comaparision.to_csv('/content/IITB_Translated_2to3.csv', index=False)

In [10]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
references = [[hindi] for hindi in hindi_sentences]
candidates = translatedEngHin
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.4507
